In [1]:
import os, cv2, copy
import numpy as np
from src.util import readJSON, parseMKV, target2cam_color, composeH, import_intrinsic_calib, export_intrinsic_calib
from visualizer import CameraPoseVisualizer
from scipy.spatial.transform import Rotation

os.chdir(globals()['_dh'][0])

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem .

ImportError: Cannot import k4a_module. DLL directory added was C:\Program Files\Azure Kinect SDK v1.2.0\sdk\windows-desktop\AMD64\release\bin. You can provide a different path containing `k4a.dll`using the environment variable `K4A_DLL_DIR`. Also make sure pyk4a was properly built.

In [ ]:
dir = "test_mkv"

out_dir = "calib"

debug = True

exportFrames = False
exportIntrinsic = False
exportRobotPose = False
estimateExtrinsic = False

In [ ]:
if exportIntrinsic:
    intrinsic_matrix, distortion_coefficients = export_intrinsic_calib(dir = dir)
else:
    intrinsic_matrix, distortion_coefficients = import_intrinsic_calib(dir = dir)

In [ ]:
from src.mkv_parser import AzureKinectMKVParser

parser = AzureKinectMKVParser(dir=dir)
parser.export_frames()
# parser.overwriteIntrinsic()

# if exportFrames:
#     parseMKV(dir = dir, exportDepth = False, debug = debug)

In [ ]:
if exportRobotPose:
    from src.abb_parser import T_ROBParser

    parser = T_ROBParser(
        dir=dir,
        parsing_method='robtarget') # or 'MoveL'
    pos, rotvec, _, _, _ = parser.base_T_tcp()
else:
    file_path = os.path.join(dir, "base_T_tcp.json")
    data = readJSON(file_path)

    pos = data.get('pos')
    rotvec = data.get('rotvec')

In [ ]:
if estimateExtrinsic:
    from src.camera_calibration import CameraCalibration

    calib = CameraCalibration(
        dir = dir,
        folder_name='ir',
        intrinsic_matrix=intrinsic_matrix,
        distortion_coefficients=distortion_coefficients,
        checkerboard_shape=(24,17),
        checkerboard_size=30.0,
        save_dir = out_dir,
        debug=debug,
        )
    calib.calibrate()